# Preprocessing delle recensioni di TripAdvisor per Roma

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import string

In [2]:
# Import dataset
df = pd.read_csv('reviews.csv', sep = ',')

# Keep reviews in italian
df_it = df[df['review_language']=='it'][['review_title', 'review_text', 'review_rating']]

print ('Numero di recensioni totali: ', df_it.shape[0], '\nNumero di recensioni totali:', df.shape[0])

Numero di recensioni totali:  81901 
Numero di recensioni totali: 223089


## Definizione delle label di output

In [3]:
def label(x):
    """The function, applied to the ratings column, returns a score between -1 and 1. 
    Specifically we do this mapping:
    1,2 ---> -1: negative
    3 ---> 0: neutro
    4,5 ---> 1: positive"""
    
    if x == 1 or  x == 2:
        return -1
    elif x == 3:
        return 0
    elif x == 4 or x == 5:
        return 1

In [4]:
# Map rating
df_it['review_rating'] = df_it['review_rating'].apply(label)

### Normalizzazione del test della review

In [5]:
# Dictionary of punctuation
punct_table = {ord(char):' ' for char in string.punctuation}

In [6]:
def normalize(x):
    """This function normalizes the review's text. In particular,
    1. remove the punctuation
    2. lower case"""
    
    # Remove punctuation
    s = x.translate(punct_table)
    s = ' '.join(w.lower() for w in s.split() if len(w)>1)
    return s

In [7]:
df_it['review_text'] = df_it['review_text'].apply(normalize)
df_it['review_title'] = df_it['review_title'].apply(normalize)

In [9]:
df_it['review_rating'].value_counts()

 1    69157
-1     6632
 0     6112
Name: review_rating, dtype: int64

## Utilizzo un vocabolario controllato per sostituire nelle review delle etichette positive, negative o neutre

In [8]:
# Import vocabolario controllato
vocabolario = pd.read_csv('sentix', sep = '\t', header = None)[[0,5]]

To each word we assign the label:

* `$POS` if 0.25<`value`<=1
* `$NEU` if -0.25<`value`<=0.25
* `$NEG` if -1<`value`<-0.25

In [10]:
def assign_label(x):
    """Returns the labels according to what written above"""
    
    if 0.25 < x <= 1:
        return "$POS"
    elif -0.25< x <= 0.25:
        return "$NEU"
    elif -1<= x <= -0.25:
        return "$NEG"

In [11]:
vocabolario[5] = vocabolario[5].apply(assign_label)

In [12]:
zip_vocabolario = {i:j for i,j in list(zip(vocabolario[0], vocabolario[5]))}

* Qui sotto riportati valori assoluti di parole positive e negative nel vocabolario a nostra disposizione

In [13]:
vocabolario[5].value_counts()

$NEG    39217
$POS    31708
$NEU     3684
Name: 5, dtype: int64

### Sostituzione nelle review

In [14]:
valid_string = [w for w in list(vocabolario[0]) if type(w)==str]
lista_parole_uniche = [w for w in list(valid_string) if len(w.split('_'))<=1]
lista_espressioni = [w for w in list(valid_string) if len(w.split('_'))>1]

Possibili parole da aggiungere:

* Impeccabile

In [15]:
def substitute_label(x):
    """Substitute word with tag pos, neg, neu"""
    l = set(lista_parole_uniche).intersection(set(x))
    return ' '.join([zip_vocabolario[i] if i in l else i for i in x])

In [16]:
df_it['review_text_token'] = df_it['review_text'].apply(lambda x: x.split())

In [17]:
df_it['review_text_flag'] = df_it['review_text_token'].apply(substitute_label)

# Ripeto le stesse operazioni sui titoli

In [18]:
df_it['review_title_token'] = df_it['review_title'].apply(lambda x: x.split())

In [19]:
df_it['review_title_flag'] = df_it['review_title_token'].apply(substitute_label)

In [22]:
df_it['review_title_token_flag'] = df_it['review_title_flag'].apply(lambda x: x.split())
df_it['review_text_token_flag'] = df_it['review_text_flag'].apply(lambda x: x.split())

In [23]:
df_it.to_csv('preprocessed_tripadvisor_final.csv', sep = ';')

# Vettorizzazione del dataset

In [34]:
df_it.head(2)

,review_title,review_text,review_rating,review_text_token,review_text_flag,review_title_token,review_title_flag,review_title_token_flag,review_text_token_flag
0,ottima posizione personale gentilissimo,ho soggiornato con la mia ragazza per notti se...,1,"[ho, soggiornato, con, la, mia, ragazza, per, ...",ho soggiornato $POS la mia $POS $POS notti $PO...,"[ottima, posizione, personale, gentilissimo]",ottima $POS personale gentilissimo,"[ottima, $POS, personale, gentilissimo]","[ho, soggiornato, $POS, la, mia, $POS, $POS, n..."
3,un bellissimo soggiorno,insieme alla mia ragazza decidemmo di passare ...,1,"[insieme, alla, mia, ragazza, decidemmo, di, p...",$POS alla mia $POS decidemmo di $POS giorni $P...,"[un, bellissimo, soggiorno]",un $POS soggiorno,"[un, $POS, soggiorno]","[$POS, alla, mia, $POS, decidemmo, di, $POS, g..."


* Creazione del vocabolario che mappa i sincoli termini ai rispettivi indici

In [27]:
total_words_title = set([w for list_words in df_it['review_title_token_flag'] for w in list_words])
total_words_text = set([w for list_words in df_it['review_text_token_flag'] for w in list_words])

In [32]:
total_words = total_words_title.union(total_words_text)
print ('Number of unique words: {}'.format(len(total_words)))

Number of unique words: 60842


In [35]:
vocabolario_index = {w:idx for idx,w in enumerate(list(total_words))}
index_vocabolario = {idx:w for w, idx in vocabolario_index.items()}

* Per il momento procedo costruendo il dataset solo per i testi

In [53]:
def replace_word_index(x):
    """Given a list of words it returns the list of indeces"""
    
    return [vocabolario_index[w] for w in x]

* Salva vocabolario

In [57]:
import json

with open('vocabolario.json', 'w') as f:
    json.dump(vocabolario_index, f)

In [54]:
df_it['review_text_token_flag_index'] = df_it['review_text_token_flag'].apply(replace_word_index)

* Salva dataset con sequenze

In [65]:
to_save = df_it[(df_it['review_rating'] == 1) | (df_it['review_rating'] == -1)][['review_rating','review_text_token_flag_index']]

In [66]:
to_save.head()

,review_rating,review_text_token_flag_index
0,1,"[15466, 51018, 24435, 13013, 52884, 24435, 244..."
3,1,"[24435, 51446, 52884, 24435, 57437, 23649, 244..."
5,1,"[15466, 51018, 24435, 13013, 52884, 24435, 244..."
8,1,"[24435, 51446, 52884, 24435, 57437, 23649, 244..."
10,1,"[7627, 10410, 53294, 23649, 53923, 19826, 6053..."


In [67]:
to_save.to_csv('dataset_sequence.csv', sep = ';')